## Task 2
### 思路一：使用改进后的Caffe2Darknet转换模型，并测试推理能力

In [1]:
import sys
sys.path.append('/root/darknet') # path of darknet
from caffe2darknet import Caffe2Darknet # 自己实现的转换

from timm.data import resolve_data_config
from timm.data.transforms_factory import create_transform
import torchvision.models as models
from PIL import Image as PILImage
from IPython.display import Image as IPythonImage

import darknet
import darknet_images

import caffe
import numpy as np

importing Jupyter notebook from caffe2darknet.ipynb


In [3]:
model_file = './resnet18.prototxt'
weight_file = './resnet18.caffemodel'
out_file_path = './out/resnet18.cfg'
c2d = Caffe2Darknet(net = model_file,
                        weight = weight_file,
                        out_file = out_file_path)

In [4]:
c2d.convert()

Loading caffemodel:  ./ResNet18/resnet18.caffemodel
0 conv1 Convolution
1 bn1_bn BatchNorm
2 bn1 Scale
3 relu ReLU
4 maxpool Pooling
5 layer1_0_conv1 Convolution
6 layer1_0_bn1_bn BatchNorm
7 layer1_0_bn1 Scale
8 layer1_0_relu ReLU
9 layer1_0_conv2 Convolution
10 layer1_0_bn2_bn BatchNorm
11 layer1_0_bn2 Scale
12 add Eltwise
14 layer1_1_conv1 Convolution
15 layer1_1_bn1_bn BatchNorm
16 layer1_1_bn1 Scale
17 layer1_1_relu ReLU
18 layer1_1_conv2 Convolution
19 layer1_1_bn2_bn BatchNorm
20 layer1_1_bn2 Scale
21 add_1 Eltwise
23 layer2_0_conv1 Convolution
24 layer2_0_bn1_bn BatchNorm
25 layer2_0_bn1 Scale
26 layer2_0_relu ReLU
27 layer2_0_conv2 Convolution
28 layer2_0_bn2_bn BatchNorm
29 layer2_0_bn2 Scale
30 layer2_0_downsample_0 Convolution
31 layer2_0_downsample_1_bn BatchNorm
32 layer2_0_downsample_1 Scale
33 add_2 Eltwise
35 layer2_1_conv1 Convolution
36 layer2_1_bn1_bn BatchNorm
37 layer2_1_bn1 Scale
38 layer2_1_relu ReLU
39 layer2_1_conv2 Convolution
40 layer2_1_bn2_bn BatchNorm
41 

## 准确性评估

### 推理

In [5]:
net = models.resnet18(pretrained=False)
config = resolve_data_config({}, model=net)
transform = create_transform(**config)
img = PILImage.open('./image/dog.jpg').convert('RGB')
tensor = transform(img).unsqueeze(0)
image = tensor.clone().numpy()

/root/miniconda3/envs/caffe_0618/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/root/miniconda3/envs/caffe_0618/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
network_path = "./out_dir/resnet18.cfg"
label_path = "./imagenet1k.data"
weights_path = "./out_dir/resnet182.weights"
network, class_names, class_colors = darknet.load_network(network_path, label_path, weights_path, 1)
result = darknet_images.image_classification(image, network, class_names)
result[:5]

max_batches: Using default '0'
learning_rate: Using default '0.001000'
momentum: Using default '0.900000'
decay: Using default '0.000100'
subdivisions: Using default '1'
policy: Using default 'constant'
   layer   filters  size/strd(dil)      input                output
   0 conv     64       7 x 7/ 2    224 x 224 x   3 ->  112 x 112 x  64 0.236 BF
   1 max                3x 3/ 2    112 x 112 x  64 ->   56 x  56 x  64 0.002 BF
Unused field: 'pad = 1'
   2 conv     64       3 x 3/ 1     56 x  56 x  64 ->   56 x  56 x  64 0.231 BF
   3 conv     64       3 x 3/ 1     56 x  56 x  64 ->   56 x  56 x  64 0.231 BF
   4 Shortcut Layer: 1,  wt = 0, wn = 0, outputs:  56 x  56 x  64 0.000 BF
   5 conv     64       3 x 3/ 1     56 x  56 x  64 ->   56 x  56 x  64 0.231 BF
   6 conv     64       3 x 3/ 1     56 x  56 x  64 ->   56 x  56 x  64 0.231 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs:  56 x  56 x  64 0.000 BF
   8 conv    128       3 x 3/ 2     56 x  56 x  64 ->   28 x  28 x 128 0.11

 Try to load cfg: ./out_dir/resnet182.cfg, weights: ./out_dir/resnet182.weights, clear = 0 
mini_batch = 1, batch = 1, time_steps = 1, train = 0 
 Try to load weights: ./out_dir/resnet182.weights 


[('cocker spaniel', 5.662687301635742),
 ("yellow lady's slipper", 5.231784820556641),
 ('umbrella', 4.674552917480469),
 ('Brittany spaniel', 4.525413513183594),
 ('balloon', 4.159382343292236)]